# Protocol Validation for PD25

This notebook contains results for creating optimal AFID annotations of the PD25 template.

In [1]:
# initialize libraries
library(plyr)
library(digest)
library(reshape2)
library(ggplot2)
library("plot3D")

In [2]:
# useful functions

# calculate the distance between two sets of coordinates
dist3D <- function(coord1, coord2) { # vector X,Y,Z
        xdist <- coord1[1] - coord2[1] # could also write as coord1$X, etc.
        ydist <- coord1[2] - coord2[2]
        zdist <- coord1[3] - coord2[3]
        return(as.numeric(sqrt(xdist^2+ydist^2+zdist^2)))
}

# calculate the pairwise distance between an array of 3D coordinates
pairwise_dist3D <- function(temp_coords) { # labeled X,Y,Z
        N <- length(temp_coords$X)
        dist_vec <- rep(0,N) # create vector
        sum_dist <- 0 # initialize to zero
        count <- 0
        for (i in 1:(N-1)) {
                for (j in (i+1):N) {
                        if (i != j) {
                                count <- count + 1
                                first_coord <- temp_coords[i,]
                                second_coord <- temp_coords[j,]
                                curr_dist <- dist3D(first_coord, second_coord)
                                sum_dist <- sum_dist + curr_dist
                                dist_vec[count] <- curr_dist
                        }
                }
        }
        return(c(as.numeric(mean(dist_vec)),as.numeric(sd(dist_vec))))
}

In [3]:
# initialize variables and load in raw fcsv data into df_raters
setwd('~/GitHub/afids-examples/template_validation/sub-PD25/data/input_afid/')

df_afid <- read.table('~/GitHub/afids-analysis/etc/afids.csv', sep=",", header=TRUE) # TODO: change path to local

df_raters <- data.frame(uid=integer(), afid=integer(),X=double(),Y=double(),Z=double(),rater=factor(),
                        subject=factor(),mri_type=factor(),session=integer(),date=integer(),
                        name=character(),description=character(),stringsAsFactors = FALSE)

csv_files <- list.files(".", "*.fcsv")

for (i in 1:length(csv_files)) {
        curr_split <- unlist(strsplit(csv_files[i],"_"))
    
        if (length(curr_split)>1) { # extract name and session data
                rater_subject <- curr_split[1]
                rater_mri_type <- curr_split[2]
                rater_name <- curr_split[3]
                rater_session <- as.numeric(curr_split[4])
                rater_date <- as.numeric(unlist(strsplit(curr_split[5],"[.]"))[1])
                rater_filename <- csv_files[i]
        }
        i
        curr_rater <- read.table(csv_files[i], header=FALSE, sep=",")
        df_rater <- data.frame(afid = 1:length(curr_rater$V1))
        
        df_rater <- cbind(df_rater,X=curr_rater[2],Y=curr_rater[3],Z=curr_rater[4],rater=rater_name,
                          subject=rater_subject,mri_type=rater_mri_type,
                          filename=rater_filename,session=rater_session,date=rater_date,name=curr_rater[12],
                          description=curr_rater[13])
        
        df_rater <- rename(df_rater, c("V2"="X","V3"="Y","V4"="Z","V12"="name","V13"="description"))
        df_raters <- rbind(df_raters,df_rater)
}

In [4]:
# mean coordinates for each landmark placement for each subject/session
df_mean <- ddply(df_raters, .(subject,afid), summarize, X=mean(X), Y=mean(Y), Z=mean(Z))

# initialize
df_raters$mean_AFLE <- NA
df_raters$outlier <- NA

df_raters$xdist <- NA
df_raters$ydist <- NA
df_raters$zdist <- NA

for (i in 1:dim(df_raters)[1]) {
        curr_rater <- df_raters[i,]
        
        # set current mean based on subject, session, fid
        mean_raters <- subset(df_mean, subject == curr_rater$subject & afid == curr_rater$afid)
        
        df_raters[i,]$xdist <- curr_rater$X - mean_raters$X
        df_raters[i,]$ydist <- curr_rater$Y - mean_raters$Y
        df_raters[i,]$zdist <- curr_rater$Z - mean_raters$Z
        curr_coords <- curr_rater[,2:4]
        mean_coords <- mean_raters[,3:5]
        df_raters[i,]$mean_AFLE <- dist3D(curr_coords, mean_coords)
        df_raters[i,]$outlier <- (df_raters[i,]$mean_AFLE > 10) # focus on true outliers (1cm+) first
}

In [5]:
# summary of findings
all_subjects <- subset(df_raters, session > 0) # ignore session 0 which was from the group tutorial
num_outliers <- sum(subset(all_subjects, outlier == TRUE)$outlier)
num_total <- length(all_subjects$outlier)

sprintf( "Total: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
        mean(all_subjects$mean_AFLE), sd(all_subjects$mean_AFLE),
        num_outliers, num_total, (num_outliers/num_total)*100 )

# summary of the outliers
summary_outliers <- subset(df_raters,outlier==TRUE)[,c("afid","subject","rater","name","description","mean_AFLE")]

# summary of results for each OASIS-1 scan that was annotated
summary_subjects_df <- ddply(df_raters, .(subject), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))
summary_afids_df <- ddply(df_raters, .(afid), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))

ddply(df_raters, .(rater), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))
#ddply(subset(df_raters, rater == 'Rater05'), .(rater,subject), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))

#subset(df_raters, mean_AFLE > 5)

[1] "Total: 2.11 +/- 4.27 mm; Outliers: 3/160 (1.88%)"

rater,mean,sd,max
AT,1.815665,2.001837,9.184762
GG,1.703725,2.384824,10.816770
HS,3.155089,8.572311,36.311558
MI,1.682358,1.933092,7.580204
PI,2.204844,2.195877,9.322041


# Post-QC

Re-analysis after quality control and filtering of outliers.

In [6]:
# now QC all the output
# some fiducials were initially mislabeled and corrected in postQC directory by consensus among raters

# initialize variables and load in raw fcsv data into df_raters
setwd('~/GitHub/afids-examples/template_validation/sub-PD25/data/input_afid_postQC/')

df_afid <- read.table('~/GitHub/afids-analysis/etc/afids.csv', sep=",", header=TRUE)

df_raters <- data.frame(uid=integer(), fid=integer(),X=double(),Y=double(),Z=double(),rater=factor(),
                        subject=factor(),mri_type=factor(),session=integer(),date=integer(),
                        name=character(),description=character(),stringsAsFactors = FALSE)

csv_files <- list.files(".", "*.fcsv")

for (i in 1:length(csv_files)) {
        curr_split <- unlist(strsplit(csv_files[i],"_"))
    
        if (length(curr_split)>1) { # extract name and session data
                rater_subject <- curr_split[1]
                rater_mri_type <- curr_split[2]
                rater_name <- curr_split[3]
                rater_session <- as.numeric(curr_split[4])
                rater_date <- as.numeric(unlist(strsplit(curr_split[5],"[.]"))[1])
                rater_filename <- csv_files[i]
        }
        i
        curr_rater <- read.table(csv_files[i], header=FALSE, sep=",")
        df_rater <- data.frame(afid = 1:length(curr_rater$V1))
        
        df_rater <- cbind(df_rater,X=curr_rater[2],Y=curr_rater[3],Z=curr_rater[4],rater=rater_name,
                          subject=rater_subject,mri_type=rater_mri_type,
                          filename=rater_filename,session=rater_session,date=rater_date,name=curr_rater[12],
                          description=curr_rater[13])
        
        df_rater <- rename(df_rater, c("V2"="X","V3"="Y","V4"="Z","V12"="name","V13"="description"))
        df_raters <- rbind(df_raters,df_rater)
}

#df_raters$uid <- seq.int(nrow(df_raters)) # add a unique identifier
#levels(df_raters$rater) <- as.numeric( substr(levels(df_raters$rater), 6, 7 ) ) # rename raters based on rater number

# mean coordinates for each landmark placement for each subject/session
df_mean <- ddply(df_raters, .(subject, afid), summarize, X=mean(X), Y=mean(Y), Z=mean(Z))

# initialize
df_raters$mean_AFLE <- NA
df_raters$outlier <- NA

df_raters$xdist <- NA
df_raters$ydist <- NA
df_raters$zdist <- NA
for (i in 1:dim(df_raters)[1]) {
        curr_rater <- df_raters[i,]
        
        # set current mean based on subject, session, fid
        mean_raters <- subset(df_mean, subject == curr_rater$subject & afid == curr_rater$afid)
        
        df_raters[i,]$xdist <- curr_rater$X - mean_raters$X
        df_raters[i,]$ydist <- curr_rater$Y - mean_raters$Y
        df_raters[i,]$zdist <- curr_rater$Z - mean_raters$Z
        curr_coords <- curr_rater[,2:4]
        mean_coords <- mean_raters[,3:5]
        df_raters[i,]$mean_AFLE <- dist3D(curr_coords, mean_coords)
        df_raters[i,]$outlier <- (df_raters[i,]$mean_AFLE > 10) # focus on true outliers (1cm+) first
}

df_raters_QC <- subset(df_raters, outlier == FALSE)

# mean coordinates for each landmark placement for each subject/session
df_mean_QC <- ddply(df_raters_QC, .(subject, afid), summarize, X=mean(X), Y=mean(Y), Z=mean(Z))

# initialize
df_raters_QC$mean_AFLE <- NA
df_raters_QC$outlier <- NA

df_raters_QC$xdist <- NA
df_raters_QC$ydist <- NA
df_raters_QC$zdist <- NA

for (i in 1:dim(df_raters_QC)[1]) {
        curr_rater <- df_raters_QC[i,]
        
        # set current mean based on subject, session, fid
        mean_raters <- subset(df_mean_QC, subject == curr_rater$subject & afid == curr_rater$afid)
        
        df_raters_QC[i,]$xdist <- curr_rater$X - mean_raters$X
        df_raters_QC[i,]$ydist <- curr_rater$Y - mean_raters$Y
        df_raters_QC[i,]$zdist <- curr_rater$Z - mean_raters$Z
        curr_coords <- curr_rater[,2:4]
        mean_coords <- mean_raters[,3:5]
        df_raters_QC[i,]$mean_AFLE <- dist3D(curr_coords, mean_coords)
        df_raters_QC[i,]$outlier <- (df_raters_QC[i,]$mean_AFLE > 10) # focus on true outliers (1cm+) first
}

# include only MR1s
#df_raters_QC <- subset(df_raters_QC, mri_session == 'MR1')

In [7]:
# summary of findings
all_subjects <- subset(df_raters_QC, session > 0) # ignore session 0 which was from the group tutorial
num_outliers <- sum(subset(all_subjects, outlier == TRUE)$outlier)
num_total <- length(all_subjects$outlier)
sprintf( "Total: %.2f +/- %.2f mm; Outliers: %d/%d (%.2f%%)",
        mean(all_subjects$mean_AFLE), sd(all_subjects$mean_AFLE),
        num_outliers, num_total, (num_outliers/num_total)*100 )

# summary of results for each OASIS-1 scan that was annotated
summary_subjects_QC_df <- ddply(df_raters_QC, .(subject), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))
summary_afids_QC_df <- ddply(df_raters_QC, .(afid), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))


summary_AFLE_QC_df <- ddply(df_raters_QC, .(afid,subject), summarize, mean_AFLE=mean(mean_AFLE), sd_AFLE=sd(mean_AFLE), max_AFLE=max(mean_AFLE))


ddply(df_raters_QC, .(rater,subject), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))
ddply(df_raters_QC, .(afid), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))

#summary_subjects_QC_df
#summary_afids_QC_df
#summary_AFLE_QC_df

[1] "Total: 1.11 +/- 0.86 mm; Outliers: 0/160 (0.00%)"

rater,subject,mean,sd,max
AT,sub-PD25,1.2191863,0.7710517,3.064117
GG,sub-PD25,0.9114921,0.6675956,2.721278
HS,sub-PD25,0.9911912,0.6832153,2.951540
MI,sub-PD25,1.1225718,1.1776903,5.189977
PI,sub-PD25,1.2880410,0.9000849,3.933191


afid,mean,sd,max
1,0.4197741,0.1452147,0.5577817
2,0.3210352,0.2414886,0.6542382
3,1.1363619,0.7946768,2.4456188
4,0.7053998,0.4722807,1.4803070
5,1.0293765,0.8600926,2.4152957
6,0.8123827,0.2721745,1.1897799
7,1.3051283,0.7106621,2.3368950
8,0.9666886,0.5157103,1.7186104
9,1.1576230,0.3819084,1.6674492
10,0.6364835,0.2324657,0.8821551


In [8]:
####################################################################
# EXPORT MEAN FIDUCIAL LOCATIONS AS FCSV FILE (with outliers filtered out)
####################################################################
setwd('~/GitHub/afids-examples/template_validation/sub-PD25/data/output_afid_postQC/')

for (curr_filename in levels(df_mean_QC$subject)) { # looping on each subject level
        curr_filename
        curr_mean <- subset(df_mean_QC, subject==curr_filename)
        curr_fcsv <- data.frame(id=paste('vtkMRMLMarkupsFiducialNode',curr_mean$afid,sep="_"),x=curr_mean$X,y=curr_mean$Y,z=curr_mean$Z,
                                       ow=0,ox=0,oy=0,oz=1,
                                       vis=1,sel=1,lock=1,label=curr_mean$afid,desc=df_afid$description,
                                       associatedNodeID='vtkMRMLScalarVolumeNode1',stringsAsFactors = FALSE)
        
        # write out table (need to use file connection approach because of header information)
        curr_fcsv_name <- paste0(curr_filename,'_afids.fcsv')
        fio <- file(curr_fcsv_name, open="wt")
        writeLines(paste('# Markups fiducial file version = 4.6'),fio)
        writeLines(paste('# CoordinateSystem = 0'),fio)
        writeLines(paste('# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID'),fio)
        write.table(curr_fcsv,fio,sep=',',quote=FALSE,col.names=FALSE,row.names=FALSE)
        close(fio)
}

In [9]:
rater_name = 'GG'
#head(df_raters_QC)
ddply(subset(df_raters_QC,rater==rater_name), .(rater), summarize, mean=mean(mean_AFLE), sd=sd(mean_AFLE), max=max(mean_AFLE))
cbind(df_afid,ddply(subset(df_raters_QC,rater==rater_name), .(name), summarize, AFLE=mean(mean_AFLE)))

rater,mean,sd,max
GG,0.9114921,0.6675956,2.721278


name,description,side,name,AFLE
1,AC,midline,1,0.3622032
2,PC,midline,2,0.1508849
3,infracollicular sulcus,midline,3,0.3984066
4,PMJ,midline,4,0.5701500
5,superior interpeduncular fossa,midline,5,0.6385174
6,R superior LMS,right,6,0.6086003
7,L superior LMS,left,7,1.6117407
8,R inferior LMS,right,8,0.5829522
9,L inferior LMS,left,9,0.7861122
10,culmen,midline,10,0.3151975


In [10]:
sessionInfo()

R version 3.5.1 (2018-07-02)
Platform: x86_64-apple-darwin14.5.0 (64-bit)
Running under: macOS High Sierra 10.13.2

Matrix products: default
BLAS: /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libBLAS.dylib
LAPACK: /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libLAPACK.dylib

locale:
[1] en_CA.UTF-8/en_CA.UTF-8/en_CA.UTF-8/C/en_CA.UTF-8/en_CA.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] plot3D_1.1.1   ggplot2_3.0.0  reshape2_1.4.3 digest_0.6.16  plyr_1.8.4    

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.17     compiler_3.5.1   pillar_1.3.0     bindr_0.1.1     
 [5] base64enc_0.1-3  tools_3.5.1      uuid_0.1-2       jsonlite_1.5    
 [9] evaluate_0.11    tibble_1.4.2     gtable_0.2.0     pkgconfig_2.0.2 
[13] rlang_0.2.1      IRdisplay_0.5.0  IRkernel_0.8.12  bindrcpp_0.2.2  
[17]